# PA Opioid Epidemic

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium

In [85]:
overdose_dat = 'Overdose_Information_Network_Data_CY_January_2018_-_Current_Monthly_County_State_Police.csv'
ovd = pd.read_csv(overdose_dat)

df = ovd.drop(['Incident Time', 'Response Time Desc', 'Revive Action Desc', 'Response Desc', 'Victim County', 'Victim OD Drug ID', 'Third Party Admin Desc', 'Administration ID', 'Dose Count', 'Dose Unit', 'Dose Desc', 'Incident ID', 'Victim State', 'Incident State', 'Victim County Latitude and Longitude', 'Incident County Latitude and Longitude', 'Victim ID', 'Victim County FIPS Code', 'Incident County FIPS Code', 'Victim County Latitude', 'Victim County Longitude'], axis=1)
df = df.rename(columns={"Survive": "Survived", "Ethnicity Desc": "Hispanic", "Accidental Exposure": "Accident", "Age Range": "Age", "Naloxone Administered": "Naloxone", "Incident County Longitude": "Longitude", "Incident County Latitude": "Latitude", "Susp OD Drug Desc": "Drug", "Gender Desc": "Gender", "Incident Date": "Date", "Incident County Name": "County"})

df['Date'] = df['Date'].replace(to_replace=r'\/', value=':', regex=True)
df['Date'] = pd.to_datetime(df['Date'], format='%m:%d:%Y')

df['Age'] = df['Age'].replace(to_replace=r'\s\-\s', value='-', regex=True)
df['Age'] = df['Age'].replace(to_replace=r'14-Oct', value='0-14')
df['Age'] = df['Age'].replace(to_replace=r'0-9', value='0-14')
df['Age'] = df['Age'].replace(to_replace=r'70-79', value='70-99')
df['Age'] = df['Age'].replace(to_replace=r'80-*', value='70-99')
df['Age'] = df['Age'].astype('category')

df['Gender'] = df['Gender'].replace(to_replace=r'Male', value='M')
df['Gender'] = df['Gender'].replace(to_replace=r'Female', value='F')
df['Gender'] = df['Gender'].replace(to_replace=r'Unknown', value='U')
df['Gender'] = df['Gender'].astype('category')

df['Hispanic'] = df['Hispanic'].replace(to_replace=r'Not Hispanic', value='F')
df['Hispanic'] = df['Hispanic'].replace(to_replace=r'Hispanic', value='T')
df['Hispanic'] = df['Hispanic'].replace(to_replace=r'Unknown', value='F')
df['Hispanic'] = df['Hispanic'].replace(to_replace=r'Mongolian', value='F')
df['Hispanic'] = df['Hispanic'].astype('category')

df['Race'] = df['Race'].replace(to_replace=r'Unknown', value='Other')
df['Race'] = df['Race'].replace(to_replace=r'Asian or Pacific Islander', value='Other')
df['Race'] = df['Race'].replace(to_replace=r'American Indian or Alaskan Native', value='Other')
df['Race'] = df['Race'].replace(to_replace=r'Other', value='O')
df['Race'] = df['Race'].replace(to_replace=r'White', value='W')
df['Race'] = df['Race'].replace(to_replace=r'Black', value='B')
df['Race'] = df['Race'].astype('category')

df['Drug'] = df['Drug'].str.lower()
df['Drug'] = df['Drug'].replace(to_replace=r'benzodiazepines (i.e.valium, xanax, ativan, etc)', value='benzodiazepines')
df['Drug'] = df['Drug'].replace(to_replace=r'buprenorphine (i.e. suboxone, sublocade, subutex, butrans, buprenex, etc)', value='buprenorphine')
df['Drug'] = df['Drug'].replace(to_replace=r'barbiturates (i.e. amytal, nembutal, etc)', value='barbiturates')
df['Drug'] = df['Drug'].replace(to_replace=r'fentanyl analog/other synthetic opioid', value='fentanyl')
df['Drug'] = df['Drug'].astype('category')

df['Accident'] = df['Accident'].replace(to_replace=r'N', value='F')
df['Accident'] = df['Accident'].replace(to_replace=r'Y', value='T')
df['Accident'] = df['Accident'].astype('category')

df['Naloxone'] = df['Naloxone'].replace(to_replace=r'N', value='F')
df['Naloxone'] = df['Naloxone'].replace(to_replace=r'Y', value='T')
df['Naloxone'] = df['Naloxone'].astype('category')

df['Survived'] = df['Survived'].replace(to_replace=r'N', value='F')
df['Survived'] = df['Survived'].replace(to_replace=r'Y', value='T')
df['Survived'] = df['Survived'].replace(to_replace=r'U', value='U')
df['Survived'] = df['Survived'].astype('category')

df['Day'] = df['Day'].astype('category')

df['County'] = df['County'].astype('category')

df = df.reset_index()
df = df.rename(columns={"index": "ID"})
df['ID'] = df['ID'] + 1

In [86]:
df.head()

,ID,Date,Day,County,Gender,Age,Race,Hispanic,Accident,Drug,Naloxone,Survived,Latitude,Longitude
0,1,2020-07-02,Thursday,York,M,50-59,W,T,F,heroin,T,T,39.921925,-76.725761
1,2,2020-09-20,Sunday,York,M,20-24,W,F,F,fentanyl,T,T,39.921925,-76.725761
2,3,2020-09-11,Friday,Union,M,50-59,W,F,F,heroin,F,F,40.964731,-77.064158
3,4,2020-07-10,Friday,York,M,40-49,W,F,F,fentanyl,T,T,39.921925,-76.725761
4,5,2020-06-20,Saturday,York,M,40-49,W,F,F,fentanyl,F,F,39.921925,-76.725761


In [87]:
counties = df.groupby(['County']).groups.keys()
county_counts = df['County'].value_counts(sort=False)
df_counties = pd.DataFrame(county_counts, counties)
df_counties = df_counties.reset_index()
df_counties = df_counties.rename(columns={"County": "Occurances", "index": "County"})

In [102]:
df_counties = df_counties.sort_values(by="Occurances", ascending=False)

In [103]:
df_counties = df_counties.reset_index(drop=True)

In [104]:
df_counties

,County,Occurances
0,Allegheny,3357
1,York,2780
2,Bucks,1946
3,Dauphin,1354
4,Luzerne,979
...,...,...
62,Warren,18
63,Greene,12
64,Forest,7
65,Sullivan,2


In [89]:
drug_types = df.groupby(['Drug']).groups.keys()
drug_counts = df['Drug'].value_counts(sort=False)
df_drugs = pd.DataFrame(drug_counts, drug_types)
df_drugs = df_drugs.reset_index()
df_drugs = df_drugs.rename(columns={"Drug": "Count", "index": "Drug"})

In [90]:
df_drugs.head()

,Drug,Count
0,alcohol,482
1,barbiturates,34
2,bath salts,15
3,benzodiazepines,464
4,buprenorphine,184


In [92]:
days = df.groupby(['Day']).groups.keys()

In [93]:
day_counts = df['Day'].value_counts(sort=False)

In [94]:
df_days = pd.DataFrame(day_counts, days)

In [96]:
df_days = df_days.reset_index()

In [97]:
df_days = df_days.rename(columns={"Day": "Count", "index": "Day"})

In [98]:
df_days

,Day,Count
0,Friday,3805
1,Monday,2994
2,Saturday,3929
3,Sunday,3274
4,Thursday,3347
5,Tuesday,3195
6,Wednesday,3164


In [ ]:
county_geo = 'PaCounty2020_01.geojson'
latitude = 40.903323 
longitude = -77.594527
pa = folium.Map(location=[latitude, longitude], zoom_start=8)
folium.GeoJson(county_geo, name="geojson").add_to(pa)
folium.LayerControl().add_to(pa)